In [1]:
import torch
import pandas as pd
from torchvision import models
import torch.nn as nn
import os
from skimage import io, transform
import numpy as np
from torchvision import transforms, utils

root_dir = "./Participants_Data_TWC/"
csv_file = root_dir+'test.csv'

In [11]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn. MaxPool2d(kernel_size=2, stride=2),
            nn. Dropout(p=0.25),
            nn. Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn. ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=0.25),
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(128 *8*8, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256,10),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(10,1),
            nn.Sigmoid()
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
#         print(x.shape)
        x = x.view(x.size(0), -1)
#         print("000", x.shape)
        x = self.linear_layers(x)
        return x

In [8]:
class WaterMarkingDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, root, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root
        self.transform = transform
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        img_name = os.path.join(self.root_dir+self.dataset.iloc[index, 0])
        img = io.imread(img_name)
        label = self.dataset.iloc[index, 1]
        label = np.array([label])
        if self.transform:
            img = self.transform(img)
        return {"image":img,"label":label}
        

In [9]:
test_dataset = WaterMarkingDataset(
    "./Participants_Data_TWC/test.csv", 
    './Participants_Data_TWC/test/',
    transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize(256),transforms.ToTensor() ])
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

In [10]:
df_test = pd.read_csv(csv_file)
model = Net()
checkpoint = torch.load('./net_50.pth', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.25, inplace=False)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0

In [12]:
device = "cuda" if torch.cuda.is_available () else "cpu"
predictions, actuals = list(), list()
for i, batch in enumerate(test_loader):
    images = batch['image'].to(device)
    target = batch['label'].to(device)
    output = model(images.float())
    output = output.detach().numpy()
    actual = target.numpy()
    actual = actual.reshape((len(actual), 1))
    output = output.round()
#     print(output, actual)
    predictions.append(output)
    actuals.append(actual)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x8192 and 6272x512)

In [ ]:
pred = np.array(predictions)
print(pred.shape)
pred = pred.reshape((6829,1))
print(pred.shape)
submission = pd.DataFrame(pred, columns=['Label'])
submission.head(10)

In [13]:
submission.to_csv('./submission_phase_5.csv')

NameError: name 'submission' is not defined